## 决策树 

https://blog.csdn.net/black_shuang/article/details/80666520


### 基本流程

决策树包含一个根节点、若干个内部节点和叶子节点，叶节点对应决策结果，其他的对应一个属性，从根节点到每个叶节点的路径对应一个判定测试序列。其目的是学习一棵泛化能力强，遵循**分而治之，divide-and-conquer**。

![](/Users/zhouchao/Documents/Books/西瓜书/Figures/4_2.png)

### 划分选择

如何划分，分支节点所包含的样本尽可能属于同一类别，即*纯度，purity*更高。

#### 信息增益

**信息熵**：样本集合D中不同类别的比例$p_k,k=1,\dots,|D|$, 则

$$Ent(D)=-\sum_{k=1}^{|D|} p_k\log_2p_k$$

越小纯度越高，最小值为0，[最大值为$\log_2|D|$](https://blog.csdn.net/feixi7358/article/details/83861858)

在某属性a上，不同其不同取值来对数据D进行划分，记为$D^v$,然后根据节点所包含样本数不同而赋予不同权重$|D^v|/|D|$,即样本数越多的分支节点的影响越大，则可以算出属性a对样本D进行划分所获的**信息增益**：

$$Gain(D,a)=Ent(D)-\sum_{v=1}^V\frac{|D^v|}{|D|} Ent(D^v)$$

**ID3**：选择信息增益最大来选择属性，*信息增益越大，则意味着属性a划分所得的纯度提升越大。*

#### 增益率

**CD4.5**：信息增益对可取值数目较多的属性有偏好，可以选择**信息增益率**进行属性选择：

$$Gain_ratio(D,a)=\frac{Gain(D,a)}{IV(a)}$$

**属性a固有值**：intrinsic value

$$IV(a)=-\sum_{v=1}^V\frac{|D^v|}{|D|}\log_2 \frac{|D^v|}{|D|}$$

但是其偏好数值较少的属性，**启发式**，先从候选划分中找出信息增益率高于平均的属性，在从中选择增益率最高的。

#### 基尼指数

**CART**决策树

